In [1]:
from livecore.messaging.moniclient import ZMQMoniClient
from livecore.util.misc import zmq_ctx 
from livecore.messaging import zmqtransport as zt

import numpy as np

import time
import json

In [2]:
SERVICE = 'sndaq_fra'
HOST = 'localhost'
PORT = 5000

### Send FRA Status Update

In [3]:
sndaq_fra_status = ['QUEUED', 'IN PROGRESS', 'SUCCESS', 'FAIL']
status = sndaq_fra_status[1]

# Using zmq_ctx, the object given as args[0] will be initialized using the provided kwargs. 
with zmq_ctx(zt.ZMQReceiverQueue, port=PORT, poll_msec=0) as listener:
    with zmq_ctx(ZMQMoniClient, svc=SERVICE, moni_host=HOST, moni_port=PORT) as sender:
        # QUEUED
        sender.send_moni(
            varname='sndaq_fr_info',
            prio=3,
            value=sndaq_fra_status[0]
        )
        time.sleep(0.1)
        
        # IN PROGRESS
        sender.send_moni(
            varname='sndaq_fr_info',
            prio=3,
            value=sndaq_fra_status[1]
        )
        time.sleep(0.1)   
        
        # SUCCESS
        sender.send_moni(
            varname='sndaq_fr_info',
            prio=3,
            value=sndaq_fra_status[2]
        )
        time.sleep(0.1)
        
    # Get messages sent before context closes
    msgs = listener.msgs()
for msg in msgs:
    print(json.dumps(msg, sort_keys=True, indent=4))

{
    "prio": 3,
    "service": "sndaq_fra",
    "t": "2023-06-26 20:40:39.774222",
    "value": "QUEUED",
    "varname": "sndaq_fr_info"
}
{
    "prio": 3,
    "service": "sndaq_fra",
    "t": "2023-06-26 20:40:39.874481",
    "value": "IN PROGRESS",
    "varname": "sndaq_fr_info"
}
{
    "prio": 3,
    "service": "sndaq_fra",
    "t": "2023-06-26 20:40:39.974684",
    "value": "SUCCESS",
    "varname": "sndaq_fr_info"
}


### Send FRA Result

The following function was taken from i3Live ([link](https://github.com/WIPACrepo/IceCube-Live/blob/4d81046d5668d9d5191f8f47be47d7b429593393/live/util/misc.py#LL48C13-L48C13))

In [4]:
import struct
import time
import binascii
from threading import Lock

class IdGen:
    '''
    This class allows to generate unique 14 char alphanumeric IDs.
    Examples: "55e5c5b7886dae", "55e5d264888771"
    These IDs are used to tag messages from LiveControl, and track
    web-generated hitspool requests.
    '''
    def __init__(self, init_increm=0):
        '''
        Note: the initial increment value can be passed to this constructor.
        '''
        self.increm = init_increm
        self.increm_lock = Lock()

    def generate_id(self):
        '''
        ID generation method. The algorithm is based on MongoDB's ObjectId
        generation, just shorter as it doesn't include the uid or machine
        identifier. Collision chances remain extremely low.
        http://docs.mongodb.org/manual/reference/object-id/
        '''
        x = struct.pack('>i', int(time.time()))
        with self.increm_lock:
            x += struct.pack('>i', self.increm)[1:4]
            self.increm = (self.increm + 1) % 0xFFFFFF
        return binascii.hexlify(x).decode()

In [5]:
# Generate Placeholder Alert
t = np.arange(-120, 240, 2) / 1e3
hits = np.abs(np.random.normal(1, 0.5, t.size))
lightcurve = [[_t, r] for _t,r in zip(t, hits)]

gen = IdGen()
request_id = gen.generate_id()
data = {
    'data':{
        "desc":"FRA Trigger",
        "server":"2ndbuild.sps.icecube.southpole.usap.gov",
        "trigger_time_ns": 12345673910111213,
        "lightcurve_start_ns": 12345670910111213,
        "lightcurve_end_ns":  12345679910111213,
        "year": 2023,
        "chi2": 5245.0,
        "chi2prob": 0.607,
        "active_channels": 5074,
        "analysis_bin_size_s": 0.002,
        "signal": 3.6,
        "signal_error": 0.59,
        "xi_uncorr": 6.1,
        "xi_corr": 4.9,
        "corrected_sig_muon_rate":161.3,
        "corrected_sig_muon_trig_rate":8.63,
        "run_number":123456,
        "request_number": 6,
        "id": request_id
    },
    "lightcurve" : {
        "2": {
            "data": lightcurve,
            "offset_ms":0},
    }
}

In [6]:
# Using zmq_ctx, the object given as args[0] will be initialized using the provided kwargs. 
with zmq_ctx(zt.ZMQReceiverQueue, port=PORT, poll_msec=100) as listener:
    with zmq_ctx(ZMQMoniClient, svc=SERVICE, moni_host=HOST, moni_port=PORT) as sender:
        sender.send_moni(
            varname='sndaq_fr_info',
            prio=3,
            value=json.dumps(data)
        )

    # Get messages sent
    msgs = listener.msgs()
for msg in msgs:
    print(json.dumps(msg, sort_keys=True, indent=4))

{
    "prio": 3,
    "service": "sndaq_fra",
    "t": "2023-06-26 20:40:40.141486",
    "value": "{\"data\": {\"desc\": \"FRA Trigger\", \"server\": \"2ndbuild.sps.icecube.southpole.usap.gov\", \"trigger_time_ns\": 12345673910111213, \"lightcurve_start_ns\": 12345670910111213, \"lightcurve_end_ns\": 12345679910111213, \"year\": 2023, \"chi2\": 5245.0, \"chi2prob\": 0.607, \"active_channels\": 5074, \"analysis_bin_size_s\": 0.002, \"signal\": 3.6, \"signal_error\": 0.59, \"xi_uncorr\": 6.1, \"xi_corr\": 4.9, \"corrected_sig_muon_rate\": 161.3, \"corrected_sig_muon_trig_rate\": 8.63, \"run_number\": 123456, \"request_number\": 6, \"id\": \"6499f7c8000000\"}, \"lightcurve\": {\"2\": {\"data\": [[-0.12, 1.0900851382303525], [-0.118, 1.1364523429152085], [-0.116, 0.7562397629731014], [-0.114, 1.141856052020671], [-0.112, 0.5936305773689177], [-0.11, 0.4404724476512799], [-0.108, 0.8865617531487731], [-0.106, 1.4520131116971982], [-0.104, 1.7873551900581446], [-0.102, 1.4007025400987652], [-